In [130]:
import json
import sys

sys.path.insert(0, '../')
from src.mlflow_utils import mlflow, get_run_list, download_run_data
from src.utils import sanitize_prediction
from tqdm.notebook import tqdm

from typing import List, Dict, Tuple

In [136]:
class ResponseDataset:
    def __init__(self, experiments: List[int]):
        runs = get_run_list(experiments)
        self._overview = {}

        for run in tqdm(runs):
            preds = download_run_data(run)
            for entry in preds['data']:
                x = GT(*entry[1:3])
                if x not in self._overview:
                    self._overview[x] = {}
                self._overview[x][run] = sanitize_prediction(entry[3])
    
    def search_title(self, title: str) -> Tuple[GT, Dict[str, str]]:
        return_val = []
        for i in self._overview:
            if title != i["title"]:
                continue
            return_val.append((i, self._overview[i]))
        return tuple(return_val)


In [137]:
test = ResponseDataset([0, 1])

  0%|          | 0/21 [00:00<?, ?it/s]

In [142]:
with open("nudossi_example.json", 'w+') as f:
    json.dump(test.search_title("Nudossi")[0], f)

In [2]:
selected_experiments = [0, 1]

In [3]:
runs = get_run_list(selected_experiments)

In [47]:
overview = {}

for run in tqdm(runs):
    preds = download_run_data(run)
    for entry in preds['data']:
        x = GT(*entry[1:3])
        if x not in overview:
            overview[x] = {}
        overview[x][run] = sanitize_prediction(entry[3])

  0%|          | 0/21 [00:00<?, ?it/s]

In [48]:
from pprint import pprint

pprint(list(overview.keys()))

[Liebe [Liebe] --- Die Liebe überwindet alle Grenzen. ===> inniges Gefühl der Zuneigung für jemanden oder für etwas,
 Liebe [Liebe] --- „Die Liebe ist in der Literatur ausreichend dargestellt worden, ihren Schilderungen kann nichts Neues hinzugefügt werden.“ ===> inniges Gefühl der Zuneigung für jemanden oder für etwas,
 Liebe [Liebe] --- „Im Mittelpunkt stand die Liebe.“ ===> inniges Gefühl der Zuneigung für jemanden oder für etwas,
 Liebe [Liebe] --- „Die Gefühle, die sie durchströmen und tragen, sind Liebe, Schutzbedürfnis und Hingabe.“ ===> inniges Gefühl der Zuneigung für jemanden oder für etwas,
 Liebe [Liebe] --- „Erst zu Beginn der Fastenzeit, nach Abreise der Schauspieler, begann ich, mich wieder ernsthaft mit meiner Liebe zu beschäftigen.“ ===> inniges Gefühl der Zuneigung für jemanden oder für etwas,
 Liebe [Liebe] --- homosexuelle Liebe, lesbische Liebe ===> sexuell oder erotisch motivierte Neigung zu jemandem oder zu einer Sache,
 Liebe [Liebe] --- Sie machten Liebe, ohne 

In [52]:
for i in overview.keys():
    if "Nudossi" != i.title:
        continue
    print(i)
    print(list(overview[i].values()))
    print()

Nudossi [Nudossi] --- „Kann ich bitte die (das, den) Nudossi haben?" ===> eine herstellerspezifische Bezeichnung für dessen Variante von Nuss-Nugat-Creme
['nicht-existierendes Wort (trinkt man wahrscheinlich Kakao)', 'Nudossi ist eine Marke von Milchmädcheneis.\n\n(Anmerkung: Nudossi ist ein historisches deutsches Eisprodukt, das nicht mehr hergestellt wird. Im Beispielsatz ist es eine Marke von Milchmädcheneis.)', 'Nudossi ist eine Marke von Kaugummi in Deutschland.\n\n(Anmerkung: Nudossi ist ein historischer Bezug, da es in den 1980er-Jahren in der DDR sehr beliebt war. Heutzutage ist die Marke nicht mehr weit verbreitet.)', 'Nudossi ist eine Marke von Schokocreme aus der DDR.\nDie Antwort wäre also: "Nudossi ist eine Marke von Schokocreme" oder einfach "DDR-Schokocreme".', 'fiktive, magische Substanz, ohne Bedeutung außerhalb des Beispiels', 'Es gibt keine bekannte Verwendung des Begriffs „Nudossi“. Möglicherweise ist dies ein Tippfehler oder ein nicht allgemein bekanntes Wort.', 'I

In [44]:
import re
from typing import List

def del_(inp: str, words: List[str]) -> str:
    for word in words:
        inp = inp.replace(word, "")
    return inp

def sanitize_context(context: str) -> str:
    return del_(context, [".", ",", "?", "!", ":", "…", ";", "‚", "‘", "«", "„"])

for gt in overview:
    test_word = gt.context_word
    if re.match(".*[^A-Za-zÀ-ž\u0370-\u03FF\u0400-\u04FF\s-].*", test_word):
        print(gt)

[[Österreich]] / da --- [[Österreich]]: „Wir haben gezeigt, dass wir das Fußballspielen nicht verlernt haben. Beim Klub machts im Moment halt mehr Spaß als beim Team. Dort herrscht eine Stimmung, da fühlt man sich unwohl!“ ===> an dieser Stelle, an diesem Ort (an der/dem sich der Sprecher/die Sprecherin befindet oder auf den dieser/diese hindeutet)
[[archaisch]] / servus --- [[archaisch]]: „quid igitur? quoniam hic servos quoque partis habet, / faciam sit, proinde ut dixi, tragicomoedia.“ (Plaut. Amph. prol. 62–63) ===> Sklave, Diener


In [33]:
preds['data'][0][3]

'starkes Gefühl der Zuneigung und Verbundenheit</s>'

In [114]:
class GT(dict):  
    def __init__(self, prompt_data, ground_truth):
        super().__init__(self, title=prompt_data['title'], context_word=sanitize_context(prompt_data['context_word']), context_sentence=prompt_data['context_sentence'], ground_truth=ground_truth)
        
    def __str__(self):
        return f"{self['context_word']} [{self['title']}] --- {self['context_sentence']} ===> {self['ground_truth']}"
    
    def __hash__(self):
        return hash((self['title'], self['context_word'], self['context_sentence'], self['ground_truth']))

In [112]:
GT({"title": "test", "context_word": "a", "context_sentence": "b"}, "This is a GT") == GT({"title": "test", "context_word": "a", "context_sentence": "b"}, "This is a GT")

True

In [108]:
GT({"title": "test", "context_word": "a", "context_sentence": "b"}, "This is a GT") 

{'title': 'test',
 'context_word': 'a',
 'context_sentence': 'b',
 'ground_truth': 'This is a GT'}